In [ ]:
!wget https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!tar -xvf VOCtrainval_06-Nov-2007.tar
!wget https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

流式输出内容被截断，只能显示最后 5000 行内容。
VOCdevkit/VOC2007/JPEGImages/001732.jpg
VOCdevkit/VOC2007/JPEGImages/001733.jpg
VOCdevkit/VOC2007/JPEGImages/001734.jpg
VOCdevkit/VOC2007/JPEGImages/001738.jpg
VOCdevkit/VOC2007/JPEGImages/001739.jpg
VOCdevkit/VOC2007/JPEGImages/001741.jpg
VOCdevkit/VOC2007/JPEGImages/001746.jpg
VOCdevkit/VOC2007/JPEGImages/001747.jpg
VOCdevkit/VOC2007/JPEGImages/001749.jpg
VOCdevkit/VOC2007/JPEGImages/001750.jpg
VOCdevkit/VOC2007/JPEGImages/001752.jpg
VOCdevkit/VOC2007/JPEGImages/001754.jpg
VOCdevkit/VOC2007/JPEGImages/001755.jpg
VOCdevkit/VOC2007/JPEGImages/001756.jpg
VOCdevkit/VOC2007/JPEGImages/001758.jpg
VOCdevkit/VOC2007/JPEGImages/001759.jpg
VOCdevkit/VOC2007/JPEGImages/001761.jpg
VOCdevkit/VOC2007/JPEGImages/001765.jpg
VOCdevkit/VOC2007/JPEGImages/001766.jpg
VOCdevkit/VOC2007/JPEGImages/001768.jpg
VOCdevkit/VOC2007/JPEGImages/001771.jpg
VOCdevkit/VOC2007/JPEGImages/001772.jpg
VOCdevkit/VOC2007/JPEGImages/001775.jpg
VOCdevkit/VOC2007/JPEGImages/001777.jpg
VOCdevkit/VOC

In [ ]:
import os
import cv2
import glob
import random
import numpy as np
from PIL import Image
import xml.etree.ElementTree as ET
OUT_SIZE = (600, 600)
SCALE = (0.3, 0.7)
FILTER_SCALE = 1 / 50

ANNOS_DIR = '/content/VOCdevkit/VOC2007/Annotations/'   #xml path
IMAGE_DIR = '/content/VOCdevkit/VOC2007/JPEGImages/'    #image path
OUT_DIR='img2007/'

# ANNO_DIR = '/content/VOCdevkit/VOC2012/Annotations/'   #xml path
# IMG_DIR = '/content/VOCdevkit/VOC2012/JPEGImages/'     #image path
# outdir='img2012/'

os.makedirs(OUT_DIR+'Annotations')
os.makedirs(OUT_DIR+'JPEGImages')

xml_head = '''<annotation>
    <folder>VOC2007</folder>
    <filename>{}</filename>.
    <source>
        <database>The VOC2007 Database</database>
        <annotation>PASCAL VOC2007</annotation>
        <image>flickr</image>
        <flickrid>325991873</flickrid>
    </source>
    <owner>
        <flickrid>null</flickrid>
        <name>null</name>
    </owner>
    <size>
        <width>{}</width>
        <height>{}</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    '''
xml_obj = '''
    <object>
        <name>{}</name>
        <pose>Rear</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{}</xmin>
            <ymin>{}</ymin>
            <xmax>{}</xmax>
            <ymax>{}</ymax>
        </bndbox>
    </object>
    '''
xml_end = '''
</annotation>'''

# xml_head = '''<annotation>
#     <folder>VOC2012</folder>
#     <filename>{}</filename>.
#     <source>
#         <database>The VOC2007 Database</database>
#         <annotation>PASCAL VOC2007</annotation>
#         <image>flickr</image>
#     </source>
#     <size>
#         <width>{}</width>
#         <height>{}</height>
#         <depth>3</depth>
#     </size>
#     <segmented>0</segmented>
#     '''
# xml_obj = '''
#     <object>        
#         <name>{}</name>
#         <pose>{}</pose>
#         <truncated>0</truncated>
#         <difficult>0</difficult>
#         <bndbox>
#             <xmin>{}</xmin>
#             <ymin>{}</ymin>
#             <xmax>{}</xmax>
#             <ymax>{}</ymax>
#         </bndbox>
#     </object>
#     '''
# xml_end = '''
# </annotation>'''


def update_image_and_anno(image_list, anno_list, indexes, out_size, scale, scale_filter=0.):
    output_img = np.zeros([out_size[0], out_size[1], 3], dtype=np.uint8)
    x_scale = scale[0] + random.random() * (scale[1] - scale[0])
    y_scale = scale[0] + random.random() * (scale[1] - scale[0])
    x_divpoint = int(x_scale * out_size[1])
    y_divpoint = int(y_scale * out_size[0])

    annos_new = []
    for i, idx in enumerate(indexes):
        image_path = image_list[idx]
        image_anno = anno_list[idx]

        image_read = cv2.imread(image_path)
        if i == 0:
            image_read = cv2.resize(image_read, (x_divpoint, y_divpoint))
            output_img[:y_divpoint, :x_divpoint, :] = image_read
            for bbox in image_anno:
                x_min = bbox[1] * x_scale
                y_min = bbox[2] * y_scale
                x_max = bbox[3] * x_scale
                y_max = bbox[4] * y_scale
                annos_new.append([bbox[0], x_min, y_min, x_max, y_max])

        elif i == 1:
            image_read = cv2.resize(image_read, (out_size[1] - x_divpoint, y_divpoint))
            output_img[:y_divpoint, x_divpoint:out_size[1], :] = image_read
            for bbox in image_anno:
                x_min = x_scale + bbox[1] * (1 - x_scale)
                y_min = bbox[2] * y_scale
                x_max = x_scale + bbox[3] * (1 - x_scale)
                y_max = bbox[4] * y_scale
                annos_new.append([bbox[0], x_min, y_min, x_max, y_max])
        elif i == 2:
            image_read = cv2.resize(image_read, (x_divpoint, out_size[0] - y_divpoint))
            output_img[y_divpoint:out_size[0], :x_divpoint, :] = image_read
            for bbox in image_anno:
                x_min = bbox[1] * x_scale
                y_min = y_scale + bbox[2] * (1 - y_scale)
                x_max = bbox[3] * x_scale
                y_max = y_scale + bbox[4] * (1 - y_scale)
                annos_new.append([bbox[0], x_min, y_min, x_max, y_max])
        else:
            image_read = cv2.resize(image_read, (out_size[1] - x_divpoint, out_size[0] - y_divpoint))
            output_img[y_divpoint:out_size[0], x_divpoint:out_size[1], :] = image_read
            for bbox in image_anno:
                x_min = x_scale + bbox[1] * (1 - x_scale)
                y_min = y_scale + bbox[2] * (1 - y_scale)
                x_max = x_scale + bbox[3] * (1 - x_scale)
                y_max = y_scale + bbox[4] * (1 - y_scale)
                annos_new.append([bbox[0], x_min, y_min, x_max, y_max])

    if 0 < scale_filter:
        annos_new = [anno for anno in annos_new if scale_filter < (anno[3] - anno[1]) and scale_filter < (anno[4] - anno[2])]

    return output_img, annos_new


def get_dataset(image_path, anno_path):
    image_paths = []
    anno_image = []
    print(anno_path)
    print(os.path.join(anno_path, '*.xml'))
    for anno_file_name in glob.glob(os.path.join(anno_path, '*.xml')):
        annos_ids = anno_file_name.split('/')[-1].split('.xml')[0] # image name
        path = os.path.join(image_path, f'{annos_ids}.jpg')
        print(path)
        image = cv2.imread(path)
        image_h, image_w, _ = image.shape
        boxes = []
        index = 0
        file_tree = ET.parse(anno_file_name)
        file_root = file_tree.getroot()
        for object in get(file_root, 'object'):
            bound_box = ckecking(object, 'boundbox', 1)
            class_ids = str(ckecking(object, 'name', 1).text)
            x_min = int(float(ckecking(bound_box, 'xmin', 1).text))
            y_min = int(float(ckecking(bound_box, 'ymin', 1).text))
            x_max = int(float(ckecking(bound_box, 'xmax', 1).text))
            y_max = int(float(ckecking(bound_box, 'ymax', 1).text))

            x_min = max(x_min, 0) / image_w
            y_min = max(y_min, 0) / image_h
            x_max = min(x_max, image_w) / image_w
            y_max = min(y_max, image_h) / image_h
            index += 1
            boxes.append([class_ids, x_min, y_min, x_max, y_max])

            if not boxes:
                continue

        image_paths.append(path)
        anno_image.append(boxes)
    return image_paths, anno_image


def get(root, name):
    return root.findall(name)
 
 
def ckecking(root, name, length):
    vars = root.findall(name)
    if len(vars) == 0:
        raise NotImplementedError('Cant find %s in %s.'%(name, root.tag))
    if length > 0 and len(vars) != length:
        raise NotImplementedError('The size of %s is wrong, %d.'%(name, length, len(vars)))
    if length == 1:
        vars = vars[0]
    return vars

In [ ]:
def main():
    index_name=1
    image_path, anno_path = get_dataset(IMAGE_DIR, ANNOS_DIR)
    index = random.sample(range(len(anno_path)), len(anno_path))
    image_num = int(len(anno_path)/4)
    print(image_num)

    for i in range(image_num):
        print('i-------',i)
        indexes=[]  # 4n-4
        indexes.append(index[4*i-4]) 
        indexes.append(index[4*i-3]) 
        indexes.append(index[4*i-2]) 
        indexes.append(index[4*i-1]) 
        image_new, anno_new = update_image_and_anno(image_path, anno_path,
                                                 indexes,
                                                 OUT_SIZE, SCALE,
                                                 scale_filter=FILTER_SCALE)
        save_image_name='aug'+str(i)+'.jpg'
        cv2.imwrite(OUT_DIR+'JPEGImages/'+save_image_name, image_new)
        object = ''
        head = xml_head.format(str(save_image_name),str(600),str(600))
        for image_anno in anno_new:
            start_p = (int(image_anno[1] * OUT_SIZE[1]), int(image_anno[2] * OUT_SIZE[0]))
            end_p = (int(image_anno[3] * OUT_SIZE[1]), int(image_anno[4] * OUT_SIZE[0]))
            cv2.rectangle(image_new, start_p, end_p, (0, 255, 0), 1, cv2.LINE_AA)
            class_name=str(image_anno[0])
            object += xml_obj.format(class_name,start_p[0],start_p[1],end_p[0],end_p[1])
        cv2.imwrite(OUT_DIR+'JPEGImages/output_'+save_image_name, image_new)
  
        image_new = cv2.cvtColor(image_new, cv2.COLOR_BGR2RGB)
        image_new = Image.fromarray(image_new.astype(np.uint8))
        index_name+=1
        xml_path=OUT_DIR+'Annotations/'+save_image_name.replace('.jpg','.xml')#xml save path
        with open(xml_path,'w') as f_xml:
            f_xml.write(head+object+xml_end)

if __name__ == '__main__':
    main()

流式输出内容被截断，只能显示最后 5000 行内容。
/content/VOCdevkit/VOC2007/JPEGImages/008385.jpg
/content/VOCdevkit/VOC2007/JPEGImages/008470.jpg
/content/VOCdevkit/VOC2007/JPEGImages/001171.jpg
/content/VOCdevkit/VOC2007/JPEGImages/007692.jpg
/content/VOCdevkit/VOC2007/JPEGImages/005396.jpg
/content/VOCdevkit/VOC2007/JPEGImages/007088.jpg
/content/VOCdevkit/VOC2007/JPEGImages/009252.jpg
/content/VOCdevkit/VOC2007/JPEGImages/003837.jpg
/content/VOCdevkit/VOC2007/JPEGImages/006096.jpg
/content/VOCdevkit/VOC2007/JPEGImages/006908.jpg
/content/VOCdevkit/VOC2007/JPEGImages/009351.jpg
/content/VOCdevkit/VOC2007/JPEGImages/009184.jpg
/content/VOCdevkit/VOC2007/JPEGImages/006348.jpg
/content/VOCdevkit/VOC2007/JPEGImages/004816.jpg
/content/VOCdevkit/VOC2007/JPEGImages/006133.jpg
/content/VOCdevkit/VOC2007/JPEGImages/008810.jpg
/content/VOCdevkit/VOC2007/JPEGImages/002785.jpg
/content/VOCdevkit/VOC2007/JPEGImages/000306.jpg
/content/VOCdevkit/VOC2007/JPEGImages/003011.jpg
/content/VOCdevkit/VOC2007/JPEGImages/0041

In [ ]:
import os, tarfile
from google.colab import files

def tar_file(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root, files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
 
  files.download(output_filename)

In [ ]:
tar_file('mosaic2007.tar', '/content/img2007')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>